In [2]:
import pandas as pd

In [3]:
df = pd.read_excel('q2.xlsx')
df

,sample,c1,c2,c3
0,1,14.20,12.85,14.15
1,2,14.30,13.65,13.90
2,3,15.00,13.40,13.65
3,4,14.60,14.20,13.60
4,5,14.55,12.75,13.20
5,6,15.15,13.35,13.20
6,7,14.60,12.50,14.05
7,8,14.55,12.80,13.80


In [4]:
# Reshape the table using melt
melted_df = pd.melt(df, id_vars=['sample'], value_vars=['c1', 'c2', 'c3'],
                   var_name='type', value_name='value')

# Display the transformed table
print(melted_df)

    sample type  value
0        1   c1  14.20
1        2   c1  14.30
2        3   c1  15.00
3        4   c1  14.60
4        5   c1  14.55
5        6   c1  15.15
6        7   c1  14.60
7        8   c1  14.55
8        1   c2  12.85
9        2   c2  13.65
10       3   c2  13.40
11       4   c2  14.20
12       5   c2  12.75
13       6   c2  13.35
14       7   c2  12.50
15       8   c2  12.80
16       1   c3  14.15
17       2   c3  13.90
18       3   c3  13.65
19       4   c3  13.60
20       5   c3  13.20
21       6   c3  13.20
22       7   c3  14.05
23       8   c3  13.80


In [5]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols


# Fit the ANOVA model
model = ols('value ~ type', data=melted_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)



            sum_sq    df          F    PR(>F)
type      8.427708   2.0  23.040765  0.000005
Residual  3.840625  21.0        NaN       NaN


In [6]:
anova_table

,sum_sq,df,F,PR(>F)
type,8.427708,2.0,23.040765,0.000005
Residual,3.840625,21.0,NaN,NaN


In [7]:
# Extract MSE (Mean Square Error) from the Residual row
mse = anova_table.loc['Residual', 'sum_sq'] / anova_table.loc['Residual', 'df']
print(f"\nMean Square Error (MSE): {mse}")


Mean Square Error (MSE): 0.18288690476190483


In [8]:
# get the t distribution
from scipy.stats import t
from math import sqrt
a = len(melted_df['type'].unique())
N = len(melted_df)
alpha = 0.05
t_val = t.ppf(1 - alpha/2, N - a)
# get the t value with 
interval_mean_groups = t_val  * sqrt(mse * (1/len(df) - 1/ N))
interval_mean = t_val  * sqrt(mse /N )

In [9]:
# confidence interval for mean
mu = melted_df['value'].mean()
print(f"\nConfidence interval for mean: ({mu - interval_mean},{mu + interval_mean})")


Confidence interval for mean: (13.651794983226946,14.014871683439722)


In [10]:
for group in melted_df['type'].unique():
    print(f"\nConfidence interval for mean of {group}:")
    group_values = melted_df[melted_df['type'] == group]['value']
    group_mean = group_values.mean()
    print(f"for group {group} the interval is => ({group_mean - interval_mean_groups},{group_mean + interval_mean_groups})")


Confidence interval for mean of c1:
for group c1 the interval is => (14.362016003188712,14.875483996811289)

Confidence interval for mean of c2:
for group c2 the interval is => (12.930766003188712,13.444233996811288)

Confidence interval for mean of c3:
for group c3 the interval is => (13.437016003188711,13.950483996811288)


In [13]:
# get the chi square value
from scipy.stats import chi2
chi_square_up = chi2.ppf(1 - alpha / 2, N  - a)
chi_square_down = chi2.ppf(alpha/2, N - a)
mse * (N - a) / (a - 1)
up_val = mse * (N - a) / chi_square_up
down_val = mse * (N - a) / chi_square_down
print(f'''
The sigma range is 
down: {down_val}
----------------------------------------------------------------  
up: {up_val}
''')


The sigma range is 
down: 0.37349637050050705
----------------------------------------------------------------  
up: 0.1082510339449627

